In [1]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow # For showing because cv2.imshow doesn't work normally
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy import signal

In [2]:
# https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/criminisi_cvpr2003.pdf

In [3]:
test_image = cv2.imread('image1.jpg', cv2.IMREAD_COLOR)
test_mask = cv2.imread('mask1.jpg', cv2.IMREAD_GRAYSCALE)

In [4]:
class inpainting():
  def __init__(self, image, mask):
    assert image.shape[:2] == mask.shape, 'Image and mask must have same shape in 0 and 1 dimensions (HWC)'

    # Original inputs (keeping just in case)
    ######################
    self.original_image = image.astype(np.float32)
      # Image is read in as BGR format, may change to RGB later
    (threshold, self.original_mask) = cv2.threshold(mask, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
      # Mask is binary [0, 255] with 255 inside target, 0 outside
    ######################

    # Values to be updated (not updated when computing priorities but must be updated after)
    ######################
    # Note that self.image and self.mask can be updated

    self.image = np.copy(self.original_image)
      # Image is read in as BGR format, may change to RGB later
    self.mask = np.copy(self.original_mask)
      # Mask is binary [0, 255] with 255 inside target, 0 outside
    self.fill_front = cv2.Canny(self.mask, 0, 255) / 255 # 1 at edges, 0 otherwise
    self.inv_mask = 1 - (np.copy(mask) / 255) # 0 inside target, 1 outside target
    ######################

    # Constant Scalars
    ######################
    self.window_size = 9
    assert self.window_size % 2 != 0 and self.window_size > 0, 'Window size must be odd and positive' # Enforce must be odd and positive
      # Default size in paper is 9 "but in practice require the user to set it to be slightly larger than the largest texture element"
    self.alpha = 255
    # self.window_area = np.square(self.window_size)
    # self.window_k = (self.window_size - 1) // 2 # Half of the window
    ######################

    # Constant Kernels
    ######################
    self.sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    self.sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    self.simple_grad_kernel_x = np.array([[0, 0, 0], [-1, 0, 1], [0, 0, 0]]) # For calculating normal of target edge
    self.simple_grad_kernel_y = np.array([[0, -1, 0], [0, 0, 0], [0, 1, 0]]) # For calculating normal of target edge
    self.ones_window = np.ones((self.window_size, self.window_size)) # Used for quick confidences calculation
    self.normalization_array = signal.convolve2d(np.ones_like(self.mask), self.ones_window, mode='same', boundary='fill', fillvalue=0) # Used for quick confidences calculation
    ######################

    # Arrays calculated using the above variables in the compute_priorities() function
    ######################
    self.grad_y = None # Defined in compute_gradients()
    self.grad_x = None # Defined in compute_gradients()
    self.edge_normal_y = None # Defined in compute_normals()
    self.edge_normal_x = None # Defined in compute_normals()
    self.C = np.copy(self.inv_mask) # 0 inside target, 1 outside target
    self.data = None # Defined in compute_data()
    self.priorities = None # Defined in compute_priorities()
    ######################

  def compute_gradients(self):
    grayscale_image = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY) # If using lab, have to change this
    self.grad_y = ndimage.convolve(grayscale_image, self.sobel_y)
    self.grad_x = ndimage.convolve(grayscale_image, self.sobel_x)

  def compute_normals(self):
    edge_grad_x = ndimage.convolve(self.inv_mask, self.simple_grad_kernel_x) # self.inv_mask is same as self.sourceRegion from git repo
    edge_grad_y = ndimage.convolve(self.inv_mask, self.simple_grad_kernel_y)
    self.edge_normal_y = -1 * edge_grad_x
    self.edge_normal_x = edge_grad_y
    normal_magnitude = np.sqrt(np.square(self.edge_normal_y) + np.square(self.edge_normal_x))
    normal_magnitude[normal_magnitude == 0] = 1 # Prevent divide by 0 by not normalizing these elements
    self.edge_normal_y /= normal_magnitude
    self.edge_normal_x /= normal_magnitude

  def compute_data(self):
    self.compute_gradients()
    self.compute_normals()
    data = (self.grad_y * self.edge_normal_y) + (self.grad_x * self.edge_normal_x)
    data = np.abs(data)
    data[data == 0] = 1e-7 # Possibly not required (repo adds 0.001 to absolute line)
    # data *= self.fill_front # self.fill_front is assumed to be 1 at edges, 0 else
    data /= self.alpha
    self.data = data

  def compute_confidences(self):
    unnormalized_confidences = signal.convolve2d(self.C, self.ones_window, mode='same', boundary='fill', fillvalue=0)
    confidences = unnormalized_confidences / self.normalization_array
    # confidences *= self.fill_front # self.fill_front is assumed to be 1 at edges, 0 else
    self.C = confidences

  def compute_priorities(self):
    self.compute_data()
    self.compute_confidences()
    self.priorities = self.C * self.data * self.fill_front # self.fill_front is assumed to be 1 at edges, 0 else

In [5]:
#  Testing fast confidences computation
a = np.array([[1, 2, 0, 0],
              [5, 3, 0, 4],
              [0, 0, 0, 7],
              [9, 3, 0, 0]])

k = np.array([[1,1,1],[1,1,1],[1,1,1]])
normalization_calc_array = np.ones_like(a)

print(signal.convolve2d(a.astype(np.float32), k, mode='same', boundary='fill'))
print(signal.convolve2d(normalization_calc_array, k, mode='same', boundary='fill', fillvalue=0))

[[11. 11.  9.  4.]
 [11. 11. 16. 11.]
 [20. 20. 17. 11.]
 [12. 12. 10.  7.]]
[[4 6 6 4]
 [6 9 9 6]
 [6 9 9 6]
 [4 6 6 4]]


In [6]:
inpainting_obj = inpainting(test_image, test_mask)
inpainting_obj.compute_priorities()

In [7]:
print(np.unique(inpainting_obj.priorities))

[0.00000000e+00 1.74083092e-10 1.74159037e-10 1.74253967e-10
 1.74272953e-10 1.74291939e-10 1.74310925e-10 1.74329911e-10
 1.74348897e-10 1.75469074e-10 1.75469074e-10 1.78070163e-10
 1.78715689e-10 1.80291531e-10 1.80291531e-10 1.82911606e-10
 1.83310313e-10 1.83557131e-10 1.83917866e-10 1.85095001e-10
 1.85095001e-10 1.87734062e-10 1.88417560e-10 1.89936444e-10
 1.89936444e-10 1.92784352e-10 1.93619738e-10 1.94891803e-10
 1.94891803e-10 1.96752436e-10 1.96752436e-10 1.96809394e-10
 1.96809394e-10 2.02467237e-10 2.03321609e-10 2.06416336e-10
 2.06416336e-10 2.10365434e-10 2.12169109e-10 2.12985509e-10
 2.14295546e-10 2.17618105e-10 2.17675063e-10 2.17675063e-10
 2.17826952e-10 2.17845938e-10 2.17864924e-10 2.17864924e-10
 2.17883910e-10 2.17921882e-10 2.17959854e-10 2.20807761e-10
 2.21624162e-10 2.22269687e-10 2.23636683e-10 2.25193539e-10
 2.26465604e-10 2.26845325e-10 2.27111130e-10 2.27452879e-10
 2.27452879e-10 2.31288061e-10 2.31971559e-10 1.94244685e-04
 1.97974494e-04 1.990376